In [1]:
from df_func.make_XY import Words_Matrix, feature_X_byChi2
from etl_func.etl_data import transform_stock_df
from args import (
    stock_df, word_df, data_time, stop_words,
    features_num, classifier_dict,
    day_arg, cutoff_arg
)

train_words_matrix = Words_Matrix(
    word_df=word_df,
    stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
    data_time=data_time,
    stop_words=stop_words
    )
train_X, train_Y = train_words_matrix.X_matrix, train_words_matrix.Y_matrix
k_feature = feature_X_byChi2(train_X, train_Y, k=features_num)


In [2]:
from sklearn.linear_model import RidgeClassifier
import datetime
RDclf = RidgeClassifier(alpha = 0.05)
RDclf.fit(train_X[k_feature], train_Y)

test_startDate = datetime.date(2020,1,1)
test_endDate = datetime.date(2020,5,1)

In [3]:
test_words_matrix = Words_Matrix(
    word_df=word_df,
    stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
    data_time=(test_startDate, test_endDate),
    stop_words=stop_words
    )
test_X, test_Y = test_words_matrix.X_matrix, test_words_matrix.Y_matrix


In [5]:

# train_words_matrix.tfidf_matrix['Date']
time_filter = train_words_matrix.XY_matrix['Date'].between(test_startDate, test_endDate)
test_data = train_words_matrix.XY_matrix[time_filter]
test_data = test_data[test_data['Label'].notna()].reset_index(drop = True)
test_data['predict_label'] = RDclf.predict(test_X[k_feature])

def eval_prediction(self, s = 1, t = 2):
    if (sum(self=="漲") >= sum(self=="跌")*s) & (sum(self=="漲") >= sum(self=="跌") + t):
        result = "漲"
    elif (sum(self=="跌") >= sum(self=="漲")*s) & (sum(self=="跌") >= sum(self=="漲") + t):
        result = "跌"
    else:
        result = 0
final_test_data = test_data.groupby(['Date']).predict_label.apply(eval_prediction).reset_index()
final_test_data = final_test_data[final_test_data["predict_label"] != 0].reset_index()
# test_data = test_data[['post_time', 'predict_label', 'label']]        #label 純粹是要用來計算pred-label多少，沒意義


,index,Date,predict_label
0,0,2020-01-01,None
1,1,2020-01-02,None
2,2,2020-01-06,None
3,3,2020-01-07,None
4,4,2020-01-08,None
5,5,2020-01-09,None
6,6,2020-01-12,None
7,7,2020-01-13,None
8,8,2020-01-14,None
9,9,2020-01-15,None


In [20]:
final_test_data[final_test_data['predict_label'].notnull()]

,index,Date,predict_label
